In [1]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from PIL import Image
from tqdm.auto import tqdm
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [2]:
folder_path=glob("../../data/ddh_standard_frame_data/*")


def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df

def DatasetCreate(folder_path):
    data_img_path=glob(folder_path+"/data/*.jpg")
    data_img_path.sort()
    standard_file_path=[]
    with open(folder_path+'/annotations.json') as f:
        annotations_json = json.load(f)
    std_frame=AnnotationsPd(annotations_json[0])['frame']
    for frame in std_frame:
        file= data_img_path[frame]
        standard_file_path.append(file)
    not_standard_file_path=list(set(data_img_path)-set(standard_file_path))   
    standard_file_path=list(set(standard_file_path))
    return standard_file_path,not_standard_file_path

def create_folder(temp_path):
    try:
        os.mkdir(temp_path)
    except FileExistsError as e:
        pass  # 에러를 무시한다.

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def image_preprocessing(image_path):
    image=Image.open(image_path)
    np_image=np.array(image)
    if np_image[520,520].mean()==0:
        image=image.crop((196,128,508,640))
        image=image.resize((312,512))
        image=expand2square(image,(0,0,0))
    else:
        image=image.crop((176,128,536,640))
        image=image.resize((360,512))
        image=expand2square(image,(0,0,0))
    image=image.resize((256,256))
    return image

In [3]:
Test_file_path=[]
Train_file_path=[]
Val_file_path=[]
Test_class=[]
Train_class=[]
Val_class=[]
for i in range(20):
    if i<=15:
        if i==7 or i==8:
            continue 
        set_name='train'
        temp_path='../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)
        create_folder(temp_path)
        standard_file_path,not_standard_file_path=DatasetCreate(folder_path[i])
        for j in range(len(standard_file_path)):
            image=image_preprocessing(standard_file_path[j])
            Train_file_path.append('/p'+str(i+1)+'/standard_'+str(j).zfill(5)+'.jpg')
            Train_class.append(1)
            image.save('../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)+'/standard_'+str(j).zfill(5)+'.jpg')
            
        for j in range(len(not_standard_file_path)):
            image=image_preprocessing(not_standard_file_path[j])
            Train_file_path.append('/p'+str(i+1)+'/not_standard_'+str(j).zfill(5)+'.jpg')
            Train_class.append(0)
            image.save('../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)+'/not_standard_'+str(j).zfill(5)+'.jpg')
    elif i<=17:
        set_name='val'
        temp_path='../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)
        create_folder(temp_path)
        standard_file_path,not_standard_file_path=DatasetCreate(folder_path[i])
        for j in range(len(standard_file_path)):
            image=image_preprocessing(standard_file_path[j])
            Val_file_path.append('/p'+str(i+1)+'/standard_'+str(j).zfill(5)+'.jpg')
            Val_class.append(1)
            image.save('../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)+'/standard_'+str(j).zfill(5)+'.jpg')
            
        for j in range(len(not_standard_file_path)):
            image=image_preprocessing(not_standard_file_path[j])
            Val_file_path.append('/p'+str(i+1)+'/not_standard_'+str(j).zfill(5)+'.jpg')
            Val_class.append(0)
            image.save('../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)+'/not_standard_'+str(j).zfill(5)+'.jpg')
    else:
        set_name='test'
        temp_path='../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)
        create_folder(temp_path)
        standard_file_path,not_standard_file_path=DatasetCreate(folder_path[i])
        for j in range(len(standard_file_path)):
            image=image_preprocessing(standard_file_path[j])
            Test_file_path.append('/p'+str(i+1)+'/standard_'+str(j).zfill(5)+'.jpg')
            Test_class.append(1)
            image.save('../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)+'/standard_'+str(j).zfill(5)+'.jpg')
            
        for j in range(len(not_standard_file_path)):
            image=image_preprocessing(not_standard_file_path[j])
            Test_file_path.append('/p'+str(i+1)+'/not_standard_'+str(j).zfill(5)+'.jpg')
            Test_class.append(0)
            image.save('../../data/standardFrame_data/scale_skip/'+set_name+'/p'+str(i+1)+'/not_standard_'+str(j).zfill(5)+'.jpg')
    
    
    

In [4]:
Test_dataframe = {'file_path' : Test_file_path, 'standard' : Test_class}
Train_dataframe = {'file_path' : Train_file_path, 'standard' : Train_class}
Validation_dataframe = {'file_path' : Val_file_path, 'standard' : Val_class}
Test_dataframe = pd.DataFrame(Test_dataframe)
Train_dataframe = pd.DataFrame(Train_dataframe)
Validation_dataframe = pd.DataFrame(Validation_dataframe)
Test_dataframe.to_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv',mode = 'w', index=False)
Train_dataframe.to_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv',mode = 'w', index=False)
Validation_dataframe.to_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv',mode = 'w', index=False)